In [1]:
import pandas as pd
import numpy as np
import pickle
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from sklearn import linear_model

In [2]:
# Load full Data
df_full = pd.read_csv('data/processed/SimpleNNData.csv', index_col=0)
y = df_full.time_to_reservation
df_full.drop(columns=['time_to_reservation'], inplace=True)
weather_var = list(df_full.columns[-22:-1])

In [5]:
df = df_full.drop(columns=['dist_to_station', 'degree']+weather_var, inplace = False)
df

,park_fuel,weekend,hour_index,eng_118I,eng_COOPER,eng_I3,eng_X1,lz_102111,lz_102121,lz_102122,...,lz_185122,lz_185123,lz_185124,lz_185131,lz_185132,lz_185141,lz_185142,lz_185143,lz_185154,lz_185203
0,43,0,6.805269,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,86,0,6.805269,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,6.805269,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,30,0,6.805269,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,32,0,6.805269,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326733,94,0,6.457874,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
326734,57,0,6.457874,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
326735,48,0,6.457874,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
326736,23,0,6.457874,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)

In [7]:
cv_select = GridSearchCV(
    estimator=linear_model.LinearRegression(), 
    param_grid={}, 
    scoring='neg_mean_squared_error', # Use MSE
    n_jobs=-2,
    return_train_score=True,
    verbose=2, 
    cv=10
)

cv_select.fit(X_train, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END .................................................... total time=  25.6s
[CV] END .................................................... total time=  30.7s
[CV] END .................................................... total time=  30.7s
[CV] END .................................................... total time=  30.6s
[CV] END .................................................... total time=  30.6s
[CV] END .................................................... total time=  30.6s
[CV] END .................................................... total time=  30.6s
[CV] END .................................................... total time=   9.1s
[CV] END .................................................... total time=   7.4s
[CV] END .................................................... total time=   7.4s


GridSearchCV(cv=10, estimator=LinearRegression(), n_jobs=-2, param_grid={},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=2)

In [13]:
r2_score(y_test,cv_select.predict(X_test))

0.16170033394182326

In [9]:
def cv(model, parameters, X_train, y_train, cf = 5):    
    """
    Performs paramter tunning using cross-validation on a specifed pipe object.
    """
    # perform cross validaiton over the input parameters
    cv_select = GridSearchCV(
        estimator=model, 
        param_grid=parameters, 
        scoring='neg_mean_squared_error', # Use MSE
        n_jobs=-2,
        return_train_score=True,
        verbose=2, 
        cv=cf
    )
    cv_select.fit(X_train, y_train)
    
    return(cv_select)

In [10]:
elastic_net_model = linear_model.ElasticNet(fit_intercept = True)

parameters = {
    'alpha': np.linspace(0.01, 1, 5),
    'l1_ratio': np.linspace(0, 1, 5)
}

elastic_net_cv = cv(elastic_net_model, parameters, X_train, y_train, cf = 5)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] END ..........................alpha=0.01, l1_ratio=0.25; total time=  18.7s
[CV] END ..........................alpha=0.01, l1_ratio=0.25; total time=  22.7s
[CV] END ..........................alpha=0.01, l1_ratio=0.25; total time=  14.5s
[CV] END ..........................alpha=0.01, l1_ratio=0.25; total time=  15.8s
[CV] END ..........................alpha=0.01, l1_ratio=0.25; total time=  16.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=  18.1s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=  14.3s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=  13.4s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=  13.5s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=  13.2s
[CV] END ..........................alpha=0.01, l1_ratio=0.75; total time=  19.3s
[CV] END ..........................alpha=0.01, 

/opt/anaconda3/envs/GNN_env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2127650.0721480963, tolerance: 474.9459093456821
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/GNN_env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2102191.6859704778, tolerance: 470.06813368214466
  model = cd_fast.enet_coordinate_descent(


[CV] END ...........................alpha=0.01, l1_ratio=0.0; total time=11.5min


/opt/anaconda3/envs/GNN_env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2130433.682704748, tolerance: 475.4333873236927
  model = cd_fast.enet_coordinate_descent(


[CV] END ...........................alpha=0.01, l1_ratio=0.0; total time=11.6min
[CV] END ...........................alpha=0.01, l1_ratio=0.0; total time=11.6min


...

/opt/anaconda3/envs/GNN_env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2230559.114592219, tolerance: 470.06813368214466
  model = cd_fast.enet_coordinate_descent(


[CV] END ............................alpha=1.0, l1_ratio=0.0; total time= 6.7min


/opt/anaconda3/envs/GNN_env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2232835.190073114, tolerance: 470.52136141300895
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/GNN_env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2225777.7209725725, tolerance: 469.1800421533928
  model = cd_fast.enet_coordinate_descent(


[CV] END ............................alpha=1.0, l1_ratio=0.0; total time= 6.6min
[CV] END ............................alpha=1.0, l1_ratio=0.0; total time= 6.7min


/opt/anaconda3/envs/GNN_env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2640769.112225657, tolerance: 590.0378905459871
  model = cd_fast.enet_coordinate_descent(


In [11]:
elastic_net_cv.predict(X_test)

array([4.8260557 , 2.37336923, 3.22173179, ..., 1.35726828, 2.15729017,
       3.02545917])

In [12]:
r2_score(y_test,elastic_net_cv.predict(X_test))

0.11895492866154489

In [14]:
elastic_net_cv.best_estimator_

ElasticNet(alpha=0.01, l1_ratio=0.0)

In [15]:
from sklearn.decomposition import PCA

In [24]:
pca = PCA(n_components=100)

In [25]:
Xp = df.filter(regex = 'lz').values

In [26]:
pca.fit(Xp)

PCA(n_components=100)

In [27]:
np.cumsum(pca.explained_variance_ratio_)

array([0.02903775, 0.05388129, 0.07257484, 0.08953093, 0.10530241,
       0.12059762, 0.13218551, 0.14353277, 0.15450656, 0.16506389,
       0.17540181, 0.18504975, 0.19464576, 0.20411271, 0.21323638,
       0.22229034, 0.23117606, 0.23996874, 0.24867303, 0.25725714,
       0.26571696, 0.27399474, 0.28211597, 0.28988929, 0.29741365,
       0.30487201, 0.31228631, 0.31965429, 0.32695363, 0.33414454,
       0.34120835, 0.34819275, 0.35507726, 0.36172459, 0.36830281,
       0.37483547, 0.3812767 , 0.38759816, 0.39379472, 0.39994173,
       0.40603396, 0.41206025, 0.41805706, 0.42399327, 0.42988186,
       0.43575522, 0.44157395, 0.44731975, 0.45303012, 0.45871893,
       0.46438029, 0.46999075, 0.47552759, 0.48098822, 0.48640892,
       0.49177495, 0.4970765 , 0.50228383, 0.50743417, 0.51252245,
       0.51752586, 0.52237667, 0.52715694, 0.53190064, 0.53661109,
       0.54129828, 0.54595757, 0.55058707, 0.55519188, 0.55974335,
       0.56429087, 0.56877174, 0.57324974, 0.57768522, 0.58206